# **Test the ByteTrack model**

In [11]:
import mmcv
from mmdet.apis import set_random_seed
cfg = mmcv.Config.fromfile('./configs/mot/bytetrack/bytetrack_retinanet_visdrone-private.py')
cfg_name = 'retina_byte_16b_2e_mixup' # 모델 이름에 맞게 변경 요망
cfg.model.detector.init_cfg.checkpoint = f'./tutorial_exps/{cfg_name}/epoch_2.pth'

cfg.work_dir = f'./tutorial_exps/final/{cfg_name}'
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.data.test.test_mode = True
print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    detector=dict(
        type='RetinaNet',
        backbone=dict(
            type='ResNet',
            depth=50,
            num_stages=4,
            out_indices=(0, 1, 2, 3),
            frozen_stages=1,
            norm_cfg=dict(type='BN', requires_grad=True),
            norm_eval=True,
            style='pytorch',
            init_cfg=dict(
                type='Pretrained', checkpoint='torchvision://resnet50')),
        neck=dict(
            type='FPN',
            in_channels=[256, 512, 1024, 2048],
            out_channels=256,
            start_level=1,
            add_extra_convs='on_input',
            num_outs=5),
        bbox_head=dict(
            type='RetinaHead',
            num_classes=5,
            in_channels=256,
            stacked_convs=4,
            feat_channels=256,
            anchor_generator=dict(
                type='AnchorGenerator',
                octave_base_scale=4,
                scales_per_octave=3,
                rat

In [12]:
from mmtrack.datasets import build_dataloader
from mmtrack.apis import init_model
from mmcv.parallel import MMDataParallel
from mmtrack.apis import single_gpu_test
from mmtrack.datasets import build_dataset

dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
    dataset,
    samples_per_gpu=1,
    workers_per_gpu=cfg.data.workers_per_gpu,
    dist=False,
    shuffle=False)

# build the model and load checkpoint
model = init_model(cfg)

model = MMDataParallel(model, device_ids=cfg.gpu_ids)
outputs = single_gpu_test(model, data_loader, out_dir = f'./tutorial_exps/final/videos/{cfg_name}', fps=15, )

eval_kwargs = cfg.get('evaluation', {}).copy()
# hard-code way to remove EvalHook args
eval_hook_args = [
    'interval', 'start', 'gpu_collect', 'save_best', 'tmpdir'
    'rule', 'by_epoch'
]
for key in eval_hook_args:
    eval_kwargs.pop(key, None)
eval_kwargs.update(dict(metric=['track']))
metric = dataset.evaluate(outputs, **eval_kwargs, resfile_path = cfg.work_dir)
print(metric)

loading annotations into memory...
Done (t=3.26s)
creating index...
index created!


2023-08-10 04:52:34,406 - mmcv - INFO - initialize RetinaNet with init_cfg {'type': 'Pretrained', 'checkpoint': './tutorial_exps/retina_byte_16b_2e_mixup/epoch_2.pth'}
INFO:mmcv:initialize RetinaNet with init_cfg {'type': 'Pretrained', 'checkpoint': './tutorial_exps/retina_byte_16b_2e_mixup/epoch_2.pth'}
2023-08-10 04:52:34,408 - mmcv - INFO - load model from: ./tutorial_exps/retina_byte_16b_2e_mixup/epoch_2.pth
INFO:mmcv:load model from: ./tutorial_exps/retina_byte_16b_2e_mixup/epoch_2.pth
2023-08-10 04:52:34,408 - mmcv - INFO - load checkpoint from local path: ./tutorial_exps/retina_byte_16b_2e_mixup/epoch_2.pth
INFO:mmcv:load checkpoint from local path: ./tutorial_exps/retina_byte_16b_2e_mixup/epoch_2.pth
2023-08-10 04:52:34,659 - mmcv - INFO - 
detector.backbone.conv1.weight - torch.Size([64, 3, 7, 7]): 
PretrainedInit: load from ./tutorial_exps/retina_byte_16b_2e_mixup/epoch_2.pth 
 
INFO:mmcv:
detector.backbone.conv1.weight - torch.Size([64, 3, 7, 7]): 
PretrainedInit: load from 

[>>>>>>>>>>>>>>>>>>>>>>>>>>] 6635/6635, 14.6 task/s, elapsed: 455s, ETA:     0sEvaluate CLEAR MOT results.
/tmp/tmpqzqfnrkt

Eval Config:
USE_PARALLEL         : False                         
NUM_PARALLEL_CORES   : 8                             
BREAK_ON_ERROR       : True                          
RETURN_ON_ERROR      : False                         
LOG_ON_ERROR         : /home/jjlee/miniconda3/envs/sia/lib/python3.9/site-packages/error_log.txt
PRINT_RESULTS        : True                          
PRINT_ONLY_COMBINED  : False                         
PRINT_CONFIG         : True                          
TIME_PROGRESS        : True                          
DISPLAY_LESS_PROGRESS : True                          
OUTPUT_SUMMARY       : True                          
OUTPUT_EMPTY_CLASSES : True                          
OUTPUT_DETAILED      : True                          
PLOT_CURVES          : True                          

MotChallenge2DBox Config:
GT_FOLDER            : /tmp/tmpqzqf

<Figure size 640x480 with 0 Axes>